<a href="https://colab.research.google.com/github/sayanbanerjee32/keyword_search/blob/main/Semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word vectors and SIF

In [1]:
%%capture
# To import kaggle datasets
!pip install --upgrade pip 
!pip install kaggle==1.5.6

In [2]:
# # import colab libraries
# from google.colab import files
# # import kaggle json to connect to kaggle user account to download datsets
# files.upload()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [4]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# download our dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p 'datasets'

 66% 17.0M/25.7M [00:01<00:00, 14.3MB/s]
100% 25.7M/25.7M [00:01<00:00, 18.1MB/s]


In [6]:
#unzip
!unzip datasets/imdb-dataset-of-50k-movie-reviews.zip -d datasets/

Archive:  datasets/imdb-dataset-of-50k-movie-reviews.zip
  inflating: datasets/IMDB Dataset.csv  


## Sentence BERT
https://evergreenllc2020.medium.com/semantic-search-engine-with-s-abbfb3cd9377

In [7]:
%%capture
!pip install sentence-transformers

In [8]:
from sentence_transformers import SentenceTransformer
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Load data and create DF

In [10]:
import pandas as pd
df = pd.read_csv('datasets/IMDB Dataset.csv')
df.shape


(50000, 2)

In [11]:
df.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive


### Preprocess and tokenise

In [14]:
import re
def fix_text(txt):
    return re.sub(r'(<br />)+','<LINEBREAK>',txt)
    # return clean(txt,
    #             fix_unicode=True,               # fix various unicode errors
    #             to_ascii=True,                  # transliterate to closest ASCII representation
    #             lower=True,                     # lowercase text
    #             no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    #             no_urls=False,                  # replace all URLs with a special token
    #             no_emails=True,                # replace all email addresses with a special token
    #             no_phone_numbers=True,         # replace all phone numbers with a special token
    #             no_numbers=True,               # replace all numbers with a special token
    #             no_digits=False,                # replace all digits with a special token
    #             no_currency_symbols=True,      # replace all currency symbols with a special token
    #             no_punct=True,                 # remove punctuations
    #             replace_with_punct="<PUNCT>",          # instead of removing punctuations you may replace them
    #             replace_with_url="<URL>",
    #             replace_with_email="<EMAIL>",
    #             replace_with_phone_number="<PHONE>",
    #             replace_with_number="<NUMBER>",
    #             replace_with_digit="0",
    #             replace_with_currency_symbol="<CUR>",
    #             lang="en"                       # set to 'de' for German special handling
    #         )

In [15]:
# nlp = spacy.load("en_core_web_sm")
# tok_text=[] # for our tokenised corpus
text = df.review.tolist()
text = [fix_text(str(i)) for i in text]
# spacy = WordTokenizer()
# toks = first(spacy([text]))
# print(coll_repr(toks, 30))

In [ ]:
# Each sentence is encoded as a 1-D vector with 78 columns
sentence_embeddings = model.encode(text)
print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))
print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])